## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
import sys
sys.path.append('../')
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..","Weather_Database","WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,axim,GH,4.8699,-2.2405,79.56,85.0,35.0,4.79,scattered clouds
1,1,bukachacha,RU,52.9833,116.9167,49.32,46.0,100.0,3.49,overcast clouds
2,2,alexandria,EG,31.2156,29.9553,61.34,77.0,0.0,8.05,clear sky
3,3,makarov,RU,48.6333,142.8000,38.05,73.0,41.0,5.48,scattered clouds
4,5,kysyl-syr,RU,63.8986,122.7617,35.10,99.0,100.0,7.38,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_df = city_data_df.loc[(min_temp <= city_data_df["Max Temp"]) & (city_data_df["Max Temp"] <= max_temp)]
filtered_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,axim,GH,4.8699,-2.2405,79.56,85.0,35.0,4.79,scattered clouds
5,6,isabela,PH,6.7085,121.9711,87.85,71.0,77.0,7.81,broken clouds
8,9,el real de santa maria,PA,8.1090,-77.7325,75.87,99.0,100.0,0.31,overcast clouds
9,12,sorong,ID,-0.8833,131.2500,82.54,78.0,100.0,8.90,overcast clouds
16,19,luanda,AO,-8.8368,13.2343,77.00,83.0,40.0,9.22,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
697,753,cayenne,GF,4.9333,-52.3333,81.09,2.0,20.0,8.99,few clouds
699,756,mahebourg,MU,-20.4081,57.7000,75.49,88.0,40.0,8.05,scattered clouds
702,759,avarua,CK,-21.2078,-159.7750,82.45,78.0,20.0,9.22,few clouds
703,760,saint george,US,37.1041,-113.5841,81.19,9.0,0.0,10.36,clear sky


In [5]:
# 4a. Determine if there are any empty rows
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              712 non-null    int64  
 1   City                 712 non-null    object 
 2   Country              708 non-null    object 
 3   Lat                  712 non-null    float64
 4   Lng                  712 non-null    float64
 5   Max Temp             712 non-null    float64
 6   Humidity             712 non-null    float64
 7   Cloudiness           712 non-null    float64
 8   Wind Speed           712 non-null    float64
 9   Current Description  712 non-null    object 
dtypes: float64(6), int64(1), object(3)
memory usage: 55.8+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

# No Empty rows but some NaN country values, better as strings
filtered_df.loc[:,"Country"] = filtered_df.Country.replace(np.nan, "NA")

C:\Users\lundk\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,axim,GH,79.56,scattered clouds,4.8699,-2.2405,
5,isabela,PH,87.85,broken clouds,6.7085,121.9711,
8,el real de santa maria,PA,75.87,overcast clouds,8.1090,-77.7325,
9,sorong,ID,82.54,overcast clouds,-0.8833,131.2500,
16,luanda,AO,77.00,scattered clouds,-8.8368,13.2343,
17,tamandare,BR,76.87,overcast clouds,-8.7597,-35.1047,
20,jeremie,HT,77.22,broken clouds,18.6500,-74.1167,
23,san patricio,US,78.69,clear sky,28.0170,-97.5169,
26,port-gentil,GA,80.47,moderate rain,-0.7193,8.7815,
29,luganville,VU,80.33,light rain,-15.5333,167.1667,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for i, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f'{row["Lat"]},{row["Lng"]}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, "Hotel Name"] = hotels["results"][0]["name"]

    except (IndexError):
        print(f"No Hotel found in City {i}: {row['City']}...")
        

No Hotel found in City 8: el real de santa maria not found...
No Hotel found in City 74: raga not found...
No Hotel found in City 90: merauke not found...
No Hotel found in City 123: yuli not found...
No Hotel found in City 128: chihuahua not found...
No Hotel found in City 168: tadine not found...
No Hotel found in City 226: ati not found...
No Hotel found in City 258: sapa not found...
No Hotel found in City 261: ouallam not found...
No Hotel found in City 265: bonfim not found...
No Hotel found in City 392: edd not found...
No Hotel found in City 415: birao not found...
No Hotel found in City 438: agadez not found...
No Hotel found in City 452: kunya not found...
No Hotel found in City 460: gogrial not found...
No Hotel found in City 480: goure not found...
No Hotel found in City 507: yaan not found...
No Hotel found in City 536: lagos not found...
No Hotel found in City 585: aitape not found...
No Hotel found in City 598: taoudenni not found...
No Hotel found in City 674: poum not 

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,axim,GH,79.56,scattered clouds,4.8699,-2.2405,Axim Beach Resort And Spa
5,isabela,PH,87.85,broken clouds,6.7085,121.9711,Isawad Resort
9,sorong,ID,82.54,overcast clouds,-0.8833,131.2500,Waigo Splash Hotel
16,luanda,AO,77.00,scattered clouds,-8.8368,13.2343,EPIC SANA Luanda
17,tamandare,BR,76.87,overcast clouds,-8.7597,-35.1047,Pousada 100 Passos


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Format Figure
figure_layout = {
    'width': '1400px',
    'height': '700px',
    'padding': '1px'
}

# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5, layout=figure_layout)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='700px', padding='1px', width='1400px'))